In [31]:
import datetime
import pandas as pd
import gymnasium as gym
import gym_anytrading
import numpy as np
import datetime
from gym_anytrading.envs import StocksEnv
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout 
from keras.optimizers import Adam
import subprocess

In [32]:
df = gym_anytrading.datasets.STOCKS_GOOGL.copy()
df.shape,df.columns

((2335, 6),
 Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object'))

In [33]:
df.head(),df.tail()

(                  Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2009-05-22  198.528534  199.524521  196.196198  196.946945  196.946945   
 2009-05-26  196.171173  202.702698  195.195190  202.382385  202.382385   
 2009-05-27  203.023026  206.136139  202.607605  202.982986  202.982986   
 2009-05-28  204.544540  206.016022  202.507507  205.405411  205.405411   
 2009-05-29  206.261261  208.823822  205.555557  208.823822  208.823822   
 
              Volume  
 Date                 
 2009-05-22  3433700  
 2009-05-26  6202700  
 2009-05-27  6062500  
 2009-05-28  5332200  
 2009-05-29  5291100  ,
                    Open         High          Low        Close    Adj Close  \
 Date                                                                          
 2018-08-23  1219.880005  1235.170044  1219.349976  1221.160034  1221.160034   
 2018-08-24  1226.000000  1237.400024  1221.420044  1236.750000  1236.7

In [35]:
from finta import TA
df['TEMA'] = TA.TEMA(df, 30) # Triple Exponential Moving Average --> in ce directie se indreapta pretul pe parcursul a 12 zile cu accent mare pe ultimele zile
df['ER'] = TA.ER(df) # Kaufman Efficiency Indicator --> confirmare a trendului (-1,1) : -1 downtrend, 1 uptrend, aprox 0 random (trend confirmation)
df['RSI'] = TA.RSI(df) # Relative Strength Index --> intre 0 si 100, 30-70 intervalul de interes, 30-0 oversold, 70-100 overbought (trend confirmation)
df['OBV'] = TA.OBV(df) # On Balance Volume --> confirmare a trendului (trend confirmation) --> se bazeaza pe volumul tranzactiilor
df['STOCH'] = TA.STOCH(df) # Stochastic Oscillator --> confirmare a trendului (trend confirmation) --> intre 0 si 100, 20-80 intervalul de interes, 20-0 oversold, 80-100 overbought
df.fillna(0, inplace=True)
df.head(20)

,Open,High,Low,Close,Adj Close,Volume,TEMA,ER,RSI,OBV,STOCH
Date,,,,,,,,,,,
2009-05-22,198.528534,199.524521,196.196198,196.946945,196.946945,3433700,196.946945,0.000000,0.000000,0.0,0.000000
2009-05-26,196.171173,202.702698,195.195190,202.382385,202.382385,6202700,201.768658,0.000000,100.000000,6202700.0,0.000000
2009-05-27,203.023026,206.136139,202.607605,202.982986,202.982986,6062500,203.105999,0.000000,100.000000,12265200.0,0.000000
2009-05-28,204.544540,206.016022,202.507507,205.405411,205.405411,5332200,204.966986,0.000000,100.000000,17597400.0,0.000000
2009-05-29,206.261261,208.823822,205.555557,208.823822,208.823822,5291100,207.561945,0.000000,100.000000,22888500.0,0.000000
2009-06-01,209.574570,215.015015,209.474472,213.493500,213.493500,6638100,211.077229,0.000000,100.000000,29526600.0,0.000000
2009-06-02,213.338333,215.195190,211.911911,214.414413,214.414413,5241900,213.421299,0.000000,100.000000,34768500.0,0.000000
2009-06-03,213.213211,216.446442,212.212219,216.041046,216.041046,7058500,215.420351,0.000000,100.000000,41827000.0,0.000000
2009-06-04,217.867874,220.840836,217.467468,220.360367,220.360367,7268900,218.225981,0.000000,100.000000,49095900.0,0.000000


In [36]:
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from environment import CustomTradingEnv
from gym_anytrading.envs import StocksEnv
from  stable_baselines3.common.vec_env import VecNormalize
import torch
torch.set_num_threads(1)
env = CustomTradingEnv(df=df,window_size=30,frame_bound=(30,int(len(df)*.8)))
print(env._get_info())
vectorized_env = DummyVecEnv([lambda: env])
# vectorized_env = VecNormalize(vectorized_env)
model = RecurrentPPO('MlpLstmPolicy', vectorized_env,  verbose=0,seed=42,
                     clip_range=0.3866909174172786,
                     gae_lambda=0.8194483147484984,
                     gamma=0.9239758194049623,
                     learning_rate=0.8089642208949643,
                     max_grad_norm=3.7567513143401103,
                     vf_coef=0.10871241805304135,
                    policy_kwargs={ "lstm_hidden_size": 128,
                    "n_lstm_layers": 5})

model.learn(total_timesteps=int(2e4),progress_bar=True)

Output()

{'total_reward': None, 'total_profit': None, 'position': None}


/home/mircea/anaconda3/envs/rl_venv/lib/python3.11/site-packages/stable_baselines3/common/callbacks.py:699: TqdmExperimentalWarning: rich is experimental/alpha
  self.pbar = tqdm(total=self.locals["total_timesteps"] - self.model.num_timesteps)


In [55]:
# evaluate the agent
from environment import CustomTradingEnv
from tqdm import tqdm
def evaluate_agent(df,model,episodes,days):
    
    test_env = CustomTradingEnv(df=df, frame_bound=(len(df)-days-30, len(df)), window_size=30)
    mean_reward, mean_profit = 0, 0
    for episode in range(episodes):
        state = test_env.reset()[0]
        while True:
            action, x = model.predict(state,deterministic=False)
            state, reward, deprecated,done, info = test_env.step(action)
            if done:
                # print(info)
                break
        mean_profit += info['total_profit']
        mean_reward += info['total_reward']
        
    mean_reward /= episodes
    mean_profit /= episodes
    
    return mean_reward, mean_profit


In [38]:
evaluate_agent(df,model,10)

(433.77001900000005, 1.5022895170061263, 2335)

In [ ]:
# # test reward function
# from tqdm import tqdm
# def evaluate_agent(df,model,episodes):
#     test_env = CustomTradingEnv(df=df, frame_bound=(int(0.8*len(df)), len(df)), window_size=30)
#     mean_reward, mean_profit = 0, 0
#     profits = []
#     rewards = []
#     for episode in tqdm(range(episodes)):
#         state = test_env.reset()[0]
#         while True:
#             action, x = model.predict(state)
#             state, reward, deprecated,done, info = test_env.step(action)
#             if done:
#                 print(info)
#                 break
#         mean_profit += info['total_profit']
#         mean_reward += info['total_reward']
#         profits.append(info['total_profit'])
#         rewards.append(info['total_reward'])
        
#     mean_reward /= episodes
#     mean_profit /= episodes
    
#     return mean_reward, mean_profit,mean_profit/mean_reward,profits,rewards

# mean_reward, mean_profit,fr,profits,rewards=evaluate_agent(df,model,episodes=100)

# profits = np.array(profits)
# rewards = np.array(rewards)
# import seaborn as sns
# sns.scatterplot(x=profits,y=rewards)

In [ ]:
from environment import CustomTradingEnv
from tqdm import tqdm
from recurrent_dqn import Agent,Utils

utils = Utils()
agent = Agent(possible_actions=[0,1],starting_mem_len=50000,max_mem_len=750000,starting_epsilon = 1, learn_rate = .0005)
env = CustomTradingEnv(df=df,window_size=30,frame_bound=(30,int(len(df)*.8)))

for i in range(100):
    frames = [] # Saving the frames for the gif
    score,info = utils.play_episode( env, agent,i, debug = True) #set debug to true for rendering
    print(info['total_profit'],info['total_reward'])
    

In [54]:
import torch
def evaluate_agent_for_dqn(df,model,days):
    test_env = CustomTradingEnv(df=df, frame_bound=(len(df)-days-30, len(df)), window_size=30)
    mean_reward, mean_profit = 0, 0
    for episode in range(1):
        state = test_env.reset()[0]
        while True:
            state_tensor = torch.tensor(np.array(state).reshape(1,30,10),dtype=torch.float32)
            action= torch.argmax(model(state_tensor),dim=1)
            state, reward, deprecated,done, info = test_env.step(action)
            if done:
                # print(info)
                break
        mean_profit += info['total_profit']
        mean_reward += info['total_reward']
        
    return mean_reward, mean_profit

In [4]:
from environment import CustomTradingEnv
from tqdm import tqdm
from recurrent_dqn import Agent,Utils

utils = Utils()
agent = Agent(possible_actions=[0,1],starting_mem_len=50000,max_mem_len=750000,starting_epsilon = 1, learn_rate = .0005)


Agent Initialized



In [65]:
import pandas as pd
from finta import TA
from environment import CustomTradingEnv

import yfinance as yf
import pandas as pd

def get_stock_data(symbol, start_date="2015-01-01", end_date="2023-12-31"):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data.to_csv(f'{symbol}.csv')
    return stock_data


def preprocess_data(df):
    # Add technical indicators
    df['TEMA'] = TA.TEMA(df, 30)
    df['ER'] = TA.ER(df)
    df['RSI'] = TA.RSI(df)
    df['OBV'] = TA.OBV(df)
    df['STOCH'] = TA.STOCH(df)

    # Fill NaN values
    df.fillna(0, inplace=True)

    return df

In [66]:
tesla_df = get_stock_data("TSLA")
eth_df = get_stock_data("ETH-USD")
btc_df = get_stock_data("BTC-USD")
google_df = get_stock_data("GOOG")
amazon_df = get_stock_data("AMZN")
apple_df = get_stock_data("AAPL")
microsoft_df = get_stock_data("MSFT")
netflix_df = get_stock_data("NFLX")
spotify_df = get_stock_data("SPOT")
paypal_df = get_stock_data("PYPL")
nike_df = get_stock_data("NKE")
disney_df = get_stock_data("DIS")
amd_df = get_stock_data("AMD")
intel_df = get_stock_data("INTC")
nvidia_df = get_stock_data("NVDA")
ibm_df = get_stock_data("IBM")
alibaba_df = get_stock_data("BABA")
uber_df = get_stock_data("UBER")
airbnb_df = get_stock_data("ABNB")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [67]:
tesla_df.shape,eth_df.shape,btc_df.shape,google_df.shape,amazon_df.shape,apple_df.shape,microsoft_df.shape,netflix_df.shape,spotify_df.shape,paypal_df.shape,nike_df.shape,disney_df.shape,amd_df.shape,intel_df.shape,nvidia_df.shape,ibm_df.shape,alibaba_df.shape,uber_df.shape,airbnb_df.shape

((2264, 6),
 (2243, 6),
 (3286, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (1447, 6),
 (2138, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (2264, 6),
 (1169, 6),
 (768, 6))

In [68]:
tesla_preprocessed = preprocess_data(tesla_df)
eth_preprocessed = preprocess_data(eth_df)
btc_preprocessed = preprocess_data(btc_df)
google_preprocessed = preprocess_data(google_df)
amazon_preprocessed = preprocess_data(amazon_df)
apple_preprocessed = preprocess_data(apple_df)
microsoft_preprocessed = preprocess_data(microsoft_df)
netflix_preprocessed = preprocess_data(netflix_df)
spotify_preprocessed = preprocess_data(spotify_df)
paypal_preprocessed = preprocess_data(paypal_df)    
nike_preprocessed = preprocess_data(nike_df)    
disney_preprocessed = preprocess_data(disney_df)
amd_preprocessed = preprocess_data(amd_df)
intel_preprocessed = preprocess_data(intel_df)
nvidia_preprocessed = preprocess_data(nvidia_df)
ibm_preprocessed = preprocess_data(ibm_df)
alibaba_preprocessed = preprocess_data(alibaba_df)
uber_preprocessed = preprocess_data(uber_df)
airbnb_preprocessed = preprocess_data(airbnb_df)

In [69]:
dfs = [tesla_preprocessed,eth_preprocessed,btc_preprocessed,google_preprocessed,amazon_preprocessed,apple_preprocessed,microsoft_preprocessed,netflix_preprocessed,spotify_preprocessed,paypal_preprocessed,nike_preprocessed,disney_preprocessed,amd_preprocessed,intel_preprocessed,nvidia_preprocessed,ibm_preprocessed,alibaba_preprocessed,uber_preprocessed,airbnb_preprocessed]
len(dfs)

19

In [70]:
df_names = ['tesla','eth','btc','google','amazon','apple','microsoft','netflix','spotify','paypal','nike','disney','amd','intel','nvidia','ibm','alibaba','uber','airbnb']

In [44]:
from environment import CustomTradingEnv

def make_env(rank,df, seed=1):
    return lambda: CustomTradingEnv(df=df,window_size=30,frame_bound=(30,int(len(df)*.8)))


In [45]:
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from environment import CustomTradingEnv
import torch
import pandas as pd
torch.set_num_threads(1)
vectorized_envs = DummyVecEnv([make_env(i,df) for i,df in enumerate(dfs)])
model = RecurrentPPO('MlpLstmPolicy', env=vectorized_envs, verbose=0,seed=42)

In [54]:
model.learn(total_timesteps=int(6e4),progress_bar=True)

Output()

In [45]:
import numpy as np
import torch
for df in dfs:
    print(evaluate_agent_for_dqn(df,agent.model))

(-10.190017700195312, 0.9394895825365609, 2264)
(460.270263671875, 1.253900230545194, 2243)
(7423.578125, 1.18857315233129, 3286)
(0.42999267578125, 1.000671041307621, 2264)
(22.680007934570312, 1.1696506758030196, 2264)
(12.539993286132812, 1.0685203077806582, 2264)
(45.220001220703125, 1.1318774043992086, 2264)
(99.92999267578125, 1.2571128844154058, 2264)
(30.460006713867188, 1.1530601788756307, 1447)
(4.450000762939453, 1.0471164942073983, 2138)
(-7.020011901855469, 0.9605430981753791, 2264)
(13.330001831054688, 1.0657589377114705, 2264)
(39.44000244140625, 1.3540304643778625, 2264)
(11.80999755859375, 1.3183394548387348, 2264)
(41.489990234375, 1.0659858996710434, 2264)
(-11.789993286132812, 0.9521080888887065, 2264)
(-8.339996337890625, 0.8871860303308777, 2264)
(-0.8899993896484375, 1.1233714370200378, 1169)
(5.529998779296875, 1.0315746646059476, 768)


In [39]:
for df in dfs:
    print(evaluate_agent(df,model,30,365))

(5.6133270263671875, 1.0246080986628678, 2264)
(1049.7305908203125, 1.744412591308461, 2243)
(25249.666015625, 2.430070951965361, 3286)
(22.691993713378906, 1.1841992800627112, 2264)
(29.76000213623047, 1.2168170683325787, 2264)
(43.56999206542969, 1.2753121292130338, 2264)
(104.6300048828125, 1.3593328822648636, 2264)
(285.0500030517578, 2.414788516871981, 2264)
(72.8800048828125, 1.6914991189907356, 1447)
(-27.10000228881836, 0.6828301484399142, 2138)
(-14.830001831054688, 0.8789913685285431, 2264)
(-16.04999542236328, 0.8251244516784136, 2264)
(46.51000213623047, 1.3792384668598172, 2264)
(8.020000457763672, 1.1371183985316793, 2264)
(307.74000549316406, 2.5774937988008575, 2264)
(21.720001220703125, 1.128343828852128, 2264)
(-43.11000061035156, 0.7318944385319239, 2264)
(34.739999771118164, 2.397214625625147, 1169)
(11.370002746582031, 1.0911692818045646, 768)


In [30]:
model.save('tuned_recurrent_ppo')

In [71]:
days_interval = [30,60,90,120,210,365,730]
with open('tuned_recurrent_ppo_performance.txt','w') as f:
    for days in days_interval:
        f.write(f'performance for {days} days of trading for 50 episodes\n')
        for i,df in enumerate(dfs):
            f.write('Company name: '+df_names[i]+'\n')
            f.write(f'{evaluate_agent(df,model,50,days)}\n')
        f.write('\n')
    f.write('\n')

ValueError: Error: Unexpected observation shape (21, 10) for Box environment, please use (30, 10) or (n_env, 30, 10) for the observation shape.

In [76]:
evaluate_agent(dfs[-1],model,50,700)

(-64.74000549316406, 0.669085023695545)

In [77]:
agent.model.eval()
days_interval = [30,60,90,120,210,365,730]
with open('recurrent_dqn_performance.txt','w') as f:
    for days in days_interval:
        f.write(f'performance for {days} days of trading\n')
        for i,df in enumerate(dfs):
            f.write('Company name: '+df_names[i]+'\n')
            f.write(f'{evaluate_agent_for_dqn(df,agent.model,days)}\n')
        f.write('\n')
    f.write('\n')

ValueError: cannot reshape array of size 210 into shape (1,30,10)

In [78]:
evaluate_agent_for_dqn(dfs[-1],agent.model,700)

(53.015106201171875, 0.2682762331500125)